#训练集的embedding

In [2]:

import os  # #
#from data_processing.Extract_Interface import Extract_Interface
import numpy as np
np.set_printoptions(suppress=True)  # 取消科学计数法输出
np.set_printoptions(threshold = np.inf)
import math
import operator
from functools import reduce
from scipy.spatial import distance_matrix
ee=math.e
import torch

In [3]:
def read_frequency_table(file_path):
    """
    读取频率分布表
    :param file_path: 频率分布表文件路径
    :return: 频率矩阵
    """
    frequency_matrix = []
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # 跳过表头
            for line in lines[1:]:
                # 跳过第一列的氨基酸名称
                row = [int(val) for val in line.strip().split('\t')[1:]]
                frequency_matrix.append(row)
    except FileNotFoundError:
        print(f"未找到频率分布表文件: {file_path}")
    return np.array(frequency_matrix)

In [4]:
MaxASA = {  
        "ALA": 129.0,  # Alanine  
        "ARG": 274.0,  # Arginine  
        "ASN": 195.0,  # Asparagine  
        "ASP": 193.0,  # Aspartate (Aspartic acid)  
        "CYS": 167.0,  # Cysteine  
        "GLU": 223.0,  # Glutamate (Glutamic acid)  
        "GLN": 225.0,  # Glutamine  
        "GLY": 104.0,  # Glycine  
        "HIS": 224.0,  # Histidine  
        "ILE": 197.0,  # Isoleucine  
        "LEU": 201.0,  # Leucine  
        "LYS": 236.0,  # Lysine  
        "MET": 224.0,  # Methionine  
        "PHE": 240.0,  # Phenylalanine  
        "PRO": 159.0,  # Proline  
        "SER": 155.0,  # Serine  
        "THR": 172.0,  # Threonine  
        "TRP": 285.0,  # Tryptophan  
        "TYR": 263.0,  # Tyrosine  
        "VAL": 174.0   # Valine  
        }  

amino_acid_order = [
    'CYS', 'SER', 'THR', 'PRO', 'ALA', 'GLY', 'ASN', 'ASP', 'GLU', 'GLN',
    'HIS', 'ARG', 'LYS', 'MET', 'ILE', 'LEU', 'VAL', 'PHE', 'TYR', 'TRP']

In [5]:
def one_of_k_encoding(x, allowable_set):
            if x not in allowable_set:
                # print(x)
                raise Exception('input {0} not in allowable set{1}:'.format(x, allowable_set))
            return list(map(lambda s: x == s, allowable_set))
    
def residue_features(residue):
            res_property1 = [1 if residue in pro_res_aliphatic_table else 0, 
                                1 if residue in pro_res_aromatic_table else 0,
                                1 if residue in pro_res_polar_neutral_table else 0,
                                1 if residue in pro_res_acidic_charged_table else 0,
                                1 if residue in pro_res_basic_charged_table else 0]
            res_property2 = [res_weight_table[residue], res_pka_table[residue], res_pkb_table[residue], res_pl_table[residue]]
            #print(np.array(res_property1 + res_property2+res_property3).shape)
            #return np.array(res_property1 + res_property2+res_property3)
            features=(res_property1 + res_property2)
            return features

In [6]:
def read_frequency_table(file_path):
    """
    读取频率分布表
    :param file_path: 频率分布表文件路径
    :return: 频率矩阵
    """
    frequency_matrix = []
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # 跳过表头
            for line in lines[1:]:
                # 跳过第一列的氨基酸名称
                row = [int(val) for val in line.strip().split('\t')[1:]]
                frequency_matrix.append(row)
    except FileNotFoundError:
        print(f"未找到频率分布表文件: {file_path}")
    return np.array(frequency_matrix)

def calculate_exposed_residues(sasa_file_path, pdb_file_path, threshold=0.5):
    """
    计算给定 SASA 文件中暴露的 CA 原子的索引
    :param sasa_file_path: SASA 文件路径
    :param pdb_file_path: PDB 文件路径
    :param threshold: RSA 阈值，默认为 0.1
    :return: 暴露 CA 原子的索引列表
    """
    sasa_data = []
    try:
        with open(sasa_file_path, 'r') as rec_file:
            for line in rec_file:
                if line.startswith('SEQ'):
                    try:
                        residue_name = line[12:15].strip()
                        sasa_value = float(line[19:26].strip())
                        sasa_data.append((residue_name, sasa_value))
                    except (ValueError, IndexError):
                        print(f"解析行 '{line.strip()}' 时出错，跳过该行。")
    except FileNotFoundError:
        print(f"未找到文件: {sasa_file_path}")

    exposed_indices = []
    ca_index = 0
    with open(pdb_file_path, 'r') as pdb_file:
        for line in pdb_file:
            if line[:4] == 'ATOM' and line[12:16].strip() == 'C':
                if ca_index < len(sasa_data):
                    exposed_indices.append(ca_index)
                ca_index += 1
    print(ca_index)
    return exposed_indices, sasa_data


def get_c_coordinates(pdb_file):
    """
    从 PDB 文件中获取所有 CA 原子的坐标
    :param pdb_file: PDB 文件路径
    :return: CA 原子坐标的 numpy 数组
    """
    ca_accord = []
    with open(pdb_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line[:4] == 'ATOM' and line[12:16].strip() == 'C':
                x = float(line[31:38])
                y = float(line[39:46])
                z = float(line[47:54])
                ca_accord.append([x, y, z])
    return np.array(ca_accord)

In [7]:
path='/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train'
save_npz_path='/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train_npz'

In [8]:
interface_rec_list = [f for f in os.listdir(path) ]
interface_rec_list.sort()

In [30]:
for num2,numm in enumerate(interface_rec_list):
    
    #if numm !='6PX5_1_X_B':
    #   continue
    print(numm)
    B_chain=numm[-3]
    A_chain=numm[-1]
    frequency_file_path='/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture2/氨基酸理化性质分布.py'
    all_path_A=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/A/all/{numm}.pdb'
    all_path_A_sasa=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/A/all/sasa/{numm}.pdb'
    glycan_path_A=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/A/glycan/{numm}.pdb'
    glycan_path_A_sasa=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/A/glycan/sasa/{numm}.pdb'
    ppi_path_A=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/A/ppi/{numm}.pdb'
    ppi_path_A_sasa=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/A/ppi/sasa/{numm}.pdb'

    #pt_A_name=f'{numm}_{A_chain}.pt'
    #A_ppt=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/A/all_esm/{pt_A_name}'
    #pt_B_name=f'{numm}_{B_chain}.pt'
    #B_ppt=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/B/all_esm/{pt_B_name}'
    frequency_file_path='/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture2/氨基酸理化性质分布.py'
    all_path_B=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/B/all/{numm}.pdb'
    all_path_B_sasa=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/B/all/sasa/{numm}.pdb'
    glycan_path_B=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/B/glycan/{numm}.pdb'
    glycan_path_B_sasa=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/B/glycan/sasa/{numm}.pdb'
    ppi_path_B=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/B/ppi/{numm}.pdb'
    ppi_path_B_sasa=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture7_sugar/train/{numm}/B/ppi/sasa/{numm}.pdb'
    esm_feature=torch.load(f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture/train/hebin_esm/{numm}_{A_chain}.pt')
    ems_33 = esm_feature['representations'][33] 
    #print(np.shape(ems_33))
    frequency_matrix = read_frequency_table(frequency_file_path)
    
    
    ##############esm_ppi特征提取-A######################
    all_resid_A=[]
    ppi_resid_A=[]
    with open(all_path_A_sasa,'r') as rec_file:
        line1 = rec_file.readlines() 
    for i in line1:
            if i[0:3] =='SEQ':
                i=i.strip()
                bb=i[12:15]
                dd=i[6:11].strip()
                cc=float(i[19:26])
                all_resid_A.append(dd) 
                
    with open(ppi_path_A_sasa,'r') as rec_file:          
        line2 = rec_file.readlines() 
    for i in line2:
            if i[0:3] =='SEQ':
                i=i.strip()
                bb=i[12:15]
                dd=i[6:11].strip()
                cc=float(i[19:26])
                ppi_resid_A.append(dd)  
    yy_A=[]
    for i,value in enumerate(all_resid_A):
        if value in ppi_resid_A:
            yy_A.append(i)
    
    
    ##############esm_ppi特征提取-B######################
    all_resid_B=[]
    ppi_resid_B=[]
    with open(all_path_B_sasa,'r') as rec_file:
        line1 = rec_file.readlines() 
    for i in line1:
            if i[0:3] =='SEQ':
                i=i.strip()
                bb=i[12:15]
                dd=i[6:11].strip()
                cc=float(i[19:26])
                all_resid_B.append(dd) 
                
    with open(ppi_path_B_sasa,'r') as rec_file:       
        line2 = rec_file.readlines() 
    for i in line2:
            if i[0:3] =='SEQ':
                i=i.strip()
                bb=i[12:15]
                dd=i[6:11].strip()
                cc=float(i[19:26])
                ppi_resid_B.append(dd)  
    yy_B=[]
    for i,value in enumerate(all_resid_B):
        if value in ppi_resid_B:
            yy_B.append(i)  
            
    #############ppi的esm拼接#####################
    
    A_len=len(all_resid_A)
    A_ems=ems_33[0:A_len]
    #print('A_ems...',np.shape(A_ems))
    B_ems=ems_33[A_len:]
    #print('B_ems...',np.shape(B_ems))
    A_ems_new=A_ems[yy_A]
    #print('A_ems_new',np.shape(A_ems_new))
    B_ems_new=B_ems[yy_B]
    #print('B_ems_new',np.shape(B_ems_new))
    esm_all=torch.cat((A_ems_new,B_ems_new),dim=0)
    
    
        ####################################归一化#############################################
    def dic_normalize(dic): #计算方法为：该值-最小值/最大值-最小值  归一化 
            #print(dic)
            max_value = dic[max(dic, key=dic.get)]
            min_value = dic[min(dic, key=dic.get)]
            #print(max_value)
            #print(min_value)
            interval = float(max_value) - float(min_value)
            #print('interval',interval)
            for key in dic.keys():
                dic[key] = round((dic[key] - min_value) / interval,4)
            dic['X'] = (max_value + min_value) / 2.0  #在字典中添加X 记录为最大值和最小值的平均值
            return dic
    # 归一化
    def normalize_pssm(value):        #position specific scoring matrix 位置特异性评分矩阵
            pssm=1/(1+ee**-value)
            pssm=round(pssm,4)
            return pssm
        
    pro_res_table = ['ALA ', 'CYS ', 'ASP ', 'GLU ', 'PHE ', 'GLY ', 'HIS ', 'ILE ', 'LYS ', 'LEU ', 'MET ', 'ASN ', 'PRO ', 'GLN ', 
                         'ARG ', 'SER ', 
                         'THR ', 'VAL ', 'TRP ', 'TYR ']
        #Whether the residue is aliphatic(脂肪族氨基酸)
    pro_res_aliphatic_table = ['ALA ', 'ILE ', 'LEU ', 'MET ', 'VAL ']
        #Whether the residue is aromatic（芳香性氨基酸）
    pro_res_aromatic_table = ['PHE ', 'TRP ', 'TYR ']
        #Whether the residue is polar neutral（中性氨基酸）
    pro_res_polar_neutral_table = ['CYS ', 'ASN ', 'GLN ', 'SER ', 'THR ']
        #Whether the residue is acidic charged（酸性氨基酸）
    pro_res_acidic_charged_table = ['ASP ', 'GLU ']
        #Whether the residue is basic charged（碱性氨基酸）
    pro_res_basic_charged_table = ['HIS ', 'LYS ', 'ARG ']
        #Residue weight（分子量）
    res_weight_table = {'ALA ': 71.08, 'CYS ': 103.15, 'ASP ': 115.09, 'GLU ': 129.12, 'PHE ': 147.18, 'GLY ': 57.05, 'HIS ': 137.14,
                                        'ILE ': 113.16, 'LYS ': 128.18, 'LEU ': 113.16, 'MET ': 131.20, 'ASN ': 114.11, 'PRO ': 97.12, 'GLN ': 128.13,
                                        'ARG ': 156.19, 'SER ': 87.08, 'THR ': 101.11, 'VAL ': 99.13, 'TRP ': 186.22, 'TYR ': 163.18}
        #The negative of the logarithm of the dissociation constant for the –COOH group（羧基的解离常数的负对数pK1）
    res_pka_table = {'ALA ': 2.34, 'CYS ': 1.96, 'ASP ': 1.88, 'GLU ': 2.19, 'PHE ': 1.83, 'GLY ': 2.34, 'HIS ': 1.82, 'ILE ': 2.36,
                                     'LYS ': 2.18, 'LEU ': 2.36, 'MET ': 2.28, 'ASN ': 2.02, 'PRO ': 1.99, 'GLN ': 2.17, 'ARG ': 2.17, 'SER ': 2.21,
                                     'THR ': 2.09, 'VAL ': 2.32, 'TRP ': 2.83, 'TYR ': 2.32}
        #The negative of the logarithm of the dissociation constant for the –NH3 group（氨基的解离常数的负对数pK2）
    res_pkb_table = {'ALA ': 9.69, 'CYS ': 10.28, 'ASP ': 9.60, 'GLU ': 9.67, 'PHE ': 9.13, 'GLY ': 9.60, 'HIS ': 9.17,
                 'ILE ': 9.60, 'LYS ': 8.95, 'LEU ': 9.60, 'MET ': 9.21, 'ASN ': 8.80, 'PRO ': 10.60, 'GLN ': 9.13,
                 'ARG ': 9.04, 'SER ': 9.15, 'THR ': 9.10, 'VAL ': 9.62, 'TRP ': 9.39, 'TYR ': 9.62}
        #The pH at the isoelectric point（等电点）
    res_pl_table = {'ALA ': 6.00, 'CYS ': 5.07, 'ASP ': 2.77, 'GLU ': 3.22, 'PHE ': 5.48, 'GLY ': 5.97, 'HIS ': 7.59,
                        'ILE ': 6.02, 'LYS ': 9.74, 'LEU ': 5.98, 'MET ': 5.74, 'ASN ': 5.41, 'PRO ': 6.30, 'GLN ': 5.65,
                        'ARG ': 10.76, 'SER ': 5.68, 'THR ': 5.60, 'VAL ': 5.96, 'TRP ': 5.89, 'TYR ': 5.96}
        #Hydrophobicity of residue (pH=2)
    res_hydrophobic_ph2_table = {'ALA ': 47, 'CYS ': 52, 'ASP ': -18, 'GLU ': 8, 'PHE ': 92, 'GLY ': 0, 'HIS ': -42, 'ILE ': 100,
                                     'LYS ': -37, 'LEU ': 100, 'MET ': 74, 'ASN ': -41, 'PRO ': -46, 'GLN ': -18, 'ARG ': -26, 'SER ': -7,
                                     'THR ': 13, 'VAL ': 79, 'TRP ': 84, 'TYR ': 49}
        #Hydrophobicity of residue (pH=7)
    res_hydrophobic_ph7_table = {'ALA ': 41, 'CYS ': 49, 'ASP ': -55, 'GLU ': -31, 'PHE ': 100, 'GLY ': 0, 'HIS ': 8, 'ILE ': 99,
                                 'LYS ': -23, 'LEU ': 97, 'MET ': 74, 'ASN ': -28, 'PRO ': -46, 'GLN ': -10, 'ARG ': -14, 'SER ': -5,
                                 'THR ': 13, 'VAL ': 76, 'TRP ': 97, 'TYR ': 63}
    res_weight_table = dic_normalize(res_weight_table)
    res_pka_table = dic_normalize(res_pka_table)
    res_pkb_table =dic_normalize(res_pkb_table)
    res_pl_table =dic_normalize(res_pl_table)
    res_hydrophobic_ph2_table =dic_normalize(res_hydrophobic_ph2_table)
    res_hydrophobic_ph7_table = dic_normalize(res_hydrophobic_ph7_table)
    
    
    
    #######################节点特征计算A################3   ppi拼接
    value1=[]
    res_list=[]
    value2=[]
    value3=[]
    res_list2=[]
    with open(all_path_A_sasa,'r') as rec_file:
            line = rec_file.readlines()
            for x in line:
                if x[0:3]=='SEQ':
                    x=x.strip()
                    resid=x[12:16]   #氨基酸的名称
                    sasa_value=x[19:26]  #溶剂可及表面积
                    dd=x[6:11].strip()   #氨基酸号
                    #print(dd,resid,sasa_value)
                    value1.append(sasa_value)
                    res_list.append(resid)
                    if dd in ppi_resid_A:
                        value3.append(sasa_value)
                        res_list2.append(resid)  #pos
    maxnum=float(max(value1))
    minnum=float(min(value1))
    internal_value=maxnum-minnum
    for x1 in value3:   #pos
        x1=float(x1)
        final_value=round((x1-minnum)/internal_value,4) 
        value2.append(final_value)  #pos
    
    rec_fea_A=[]
    if len(value2)==len(res_list2): 
        for m in range(len(value2)):
            sasa=[value2[m]] #sasa为归一化后的值
            pro_seq=[res_list2[m]]  #氨基酸名称
            resid=res_list2[m]
            pro_hot=np.zeros((1,len(pro_res_table)))  
            for i in range(len(pro_seq)):  #i=1
                pro_hot[i,] = one_of_k_encoding(pro_seq[i], pro_res_table)
            pro_hot1=reduce(operator.add, pro_hot)  #将二维将至一维
            pro_hot1=pro_hot1.tolist()  #将矩阵转化为列表，以便下面的特征合并
            rec_feature=residue_features(resid)
            rec_fea_A.append(pro_hot1+rec_feature+sasa) 
        rec_fea_A=np.array(rec_fea_A)   
    rec_fea_tensor_A = torch.from_numpy(rec_fea_A)
        
    #######################节点特征计算B################3   ppi拼接
    
    value1_B=[]
    res_list_B=[]
    value2_B=[]
    value3_B=[]
    res_list2_B=[]
    with open(all_path_B_sasa,'r') as rec_file:
            line = rec_file.readlines()
            for x in line:
                if x[0:3] =='SEQ':
                    x=x.strip()
                    resid=x[12:16]   #氨基酸的名称
                    sasa_value=x[19:26]  #溶剂可及表面积
                    dd=x[6:11].strip()   #氨基酸号
                    #print(dd,resid,sasa_value)
                    value1_B.append(sasa_value)
                    res_list_B.append(resid)
                    
                    if dd in ppi_resid_B:
                        value3_B.append(sasa_value)
                        res_list2_B.append(resid)
    maxnum=float(max(value1_B))
    minnum=float(min(value1_B))
    internal_value=maxnum-minnum
    for x1 in value3_B:  #sasa
        x1=float(x1)
        final_value=round((x1-minnum)/internal_value,4) 
        value2_B.append(final_value) 
    
    rec_fea_B=[]
    if len(value2_B)==len(res_list2_B): 
        for m in range(len(value2_B)):
            sasa=[value2_B[m]] #sasa为归一化后的值
            pro_seq=[res_list2_B[m]]  #氨基酸名称
            resid=res_list2_B[m]
            pro_hot=np.zeros((1,len(pro_res_table)))  
            for i in range(len(pro_seq)):  #i=1
                pro_hot[i,] = one_of_k_encoding(pro_seq[i], pro_res_table)
            pro_hot1=reduce(operator.add, pro_hot)  #将二维将至一维
            pro_hot1=pro_hot1.tolist()  #将矩阵转化为列表，以便下面的特征合并
            rec_feature=residue_features(resid)
            rec_fea_B.append(pro_hot1+rec_feature+sasa) 
        rec_fea_B=np.array(rec_fea_B)   
    rec_fea_tensor_B = torch.from_numpy(rec_fea_B)
  
    rec_lig_feature = torch.cat((rec_fea_tensor_A, rec_fea_tensor_B), dim=0)
    #print('初步合并后的特征矩阵大小为：',np.shape(rec_lig_feature))
    
    #print(np.shape(esm_all))
    final_feature = torch.cat((rec_lig_feature, esm_all), dim=1)
    #('最后合并后的特征矩阵的大小为：',np.shape(final_feature))
  
    #################################################3333距离特征计算##############################################
    ca_accord_rec_A=[]
    residues_A = {} 
    a=0
    with open(ppi_path_A, 'r') as file:  
        for line in file:  
            if line.startswith('ATOM'):  
                            # 解析PDB行的关键信息  
                            atom_name = line[12:16].strip()  
                            residue_name = line[17:20].strip()  
                            chain_id = line[21].strip()  
                            residue_number = (line[22:27].strip())  
                            x = float(line[30:38].strip())  
                            y = float(line[38:46].strip())  
                            z = float(line[46:54].strip())  
                            
                            if atom_name == 'CA': ##太离谱了，有的没有CA有的多个CA,....................................... 
                                # 如果这是当前氨基酸的第一个CA，则存储它  
                                a+=1
                                if (chain_id, residue_number) not in residues_A:  
                                    residues_A[(chain_id, residue_number)] = (x, y, z)  
    for (chain_id, residue_number), (x, y, z) in residues_A.items():  
            ca_accord_rec_A.append([x]+[y]+[z])
            #print(residues)            
            #print(ca_accord_rec)
            a+=1
    ca_accord_rec_A=np.array(ca_accord_rec_A)
    dis_rec_rec_A = distance_matrix(ca_accord_rec_A, ca_accord_rec_A)
        #算出来的距离进行处理，受体-受体和配体-配体之间的距离小于4.5为1，受体-配体之间的距离小于10为1
    for ii in np.nditer(dis_rec_rec_A,op_flags=['readwrite']):
            if ii >8.0:
                ii[...]=0
            else:
                ii[...]=1
                
    a1=rec_fea_tensor_A.shape
    a2=dis_rec_rec_A.shape
    a11=a1[0]
    a22=a2[0]
    #print(f'A链的特征矩阵的大小为:{a1},A链的邻接矩阵的小为{a2}')
    if a11!=a22:
            print(numm,'A链有问题###################################################################')
            print(a1)
            print(a2)
    sA=np.shape(dis_rec_rec_A)
    lenth1=sA[0]
    width1=sA[1]
    
    ca_accord_rec_B=[]
    residues_B = {} 
    a=0
    with open(ppi_path_B, 'r') as file:  
        for line in file:  
            if line.startswith('ATOM'):  
                            # 解析PDB行的关键信息  
                atom_name = line[12:16].strip()  
                residue_name = line[17:20].strip()  
                chain_id = line[21].strip()  
                residue_number = (line[22:27].strip())  
                x = float(line[30:38].strip())  
                y = float(line[38:46].strip())  
                z = float(line[46:54].strip())  
                if atom_name == 'CA': ##太离谱了，有的没有CA有的多个CA,....................................... 
                                # 如果这是当前氨基酸的第一个CA，则存储它  
                                a+=1
                                if (chain_id, residue_number) not in residues_B:  
                                    residues_B[(chain_id, residue_number)] = (x, y, z)  
    for (chain_id, residue_number), (x, y, z) in residues_B.items():  
            ca_accord_rec_B.append([x]+[y]+[z])
            #print(residues)            
            #print(ca_accord_rec)
            a+=1
    ca_accord_rec_B=np.array(ca_accord_rec_B)
    dis_rec_rec_B = distance_matrix(ca_accord_rec_B, ca_accord_rec_B)
        #算出来的距离进行处理，受体-受体和配体-配体之间的距离小于4.5为1，受体-配体之间的距离小于10为1
    for ii in np.nditer(dis_rec_rec_B,op_flags=['readwrite']):
            if ii >8.0:
                ii[...]=0
            else:
                ii[...]=1
    a1=rec_fea_tensor_B.shape
    a2=dis_rec_rec_B.shape
    a11=a1[0]
    a22=a2[0]
    #print(f'B链的特征矩阵的大小为:{a1},B链的邻接矩阵的小为{a2}')
    if a11!=a22:
            print(numm,'B链有问题###################################################################')
            #print(a1)
            #print(a2)
    sB=np.shape(dis_rec_rec_B)
    lenth2=sB[0]
    width2=sB[1]
    #print(f'lenth2的长度为：',lenth2,'lenth1的长度为：',lenth1)
    combined_matrix = np.zeros((lenth1+lenth2, lenth1+lenth2))
    combined_matrix[:lenth1, :lenth1] = dis_rec_rec_A
    combined_matrix[lenth1:, lenth1:] = dis_rec_rec_B
    
    #print('合并后的邻接矩阵大小combined_matrix为：',np.shape(combined_matrix))
    
    exposed_rec1, sasa_data_rec1 = calculate_exposed_residues(ppi_path_A_sasa, ppi_path_A)
    exposed_rec2, sasa_data_rec2 = calculate_exposed_residues(ppi_path_B_sasa, ppi_path_B)
    #frequency_matrix = read_frequency_table(frequency_file_path)
    adj_matrix = np.zeros((len(exposed_rec1),len(exposed_rec2)))
    for i,idx1 in enumerate(exposed_rec1):
        residue_name1 = sasa_data_rec1[idx1][0]
        for j,idx2 in enumerate(exposed_rec2):
            residue_name2 = sasa_data_rec2[idx2][0]
            try:
                amino_acid_index1 = amino_acid_order.index(residue_name1)
                amino_acid_index2 = amino_acid_order.index(residue_name2)
            
                if frequency_matrix[amino_acid_index1,amino_acid_index2] > 0:
                    adj_matrix[i,j] = 1
            except ValueError:
                continue
            
    for i ,idx1 in enumerate(exposed_rec1): 
        for j ,idx2 in enumerate(exposed_rec2):
            combined_matrix[idx1,lenth1+idx2] =adj_matrix[i,j]   
            combined_matrix[lenth1 + idx2, idx1] = adj_matrix[i, j]   
    
    
    #print('合并后的邻接矩阵大小combined_matrix2为：',np.shape(combined_matrix))
    ################################标签计算-all ####################################################
    all_target=[]
    with open(ppi_path_A_sasa,'r') as tar_file:
        line=tar_file.readlines()
        for x in line:
            if x[0:3] =='SEQ':
                x=x.strip()
                resid=x[12:16]   #氨基酸的名称
                dd=x[6:11].strip()   #氨基酸号
                chain_id=x[4]
                all_target.append((chain_id,dd,resid))
                
    with open(ppi_path_B_sasa,'r') as tar_file2:
        line=tar_file2.readlines()
        for x in line:
            if x[0:3] =='SEQ':
                x=x.strip()
                resid=x[12:16]   #氨基酸的名称
                dd=x[6:11].strip()   #氨基酸号
                chain_id=x[4]
                all_target.append((chain_id,dd,resid))
    pos_target=[]   #glycan——B
    with open(glycan_path_B_sasa,'r') as pos_file:
        line=pos_file.readlines()
        for x in line:
            if x[0:3] =='SEQ':
                x=x.strip()
                resid=x[12:16]   #氨基酸的名称
                dd=x[6:11].strip()   #氨基酸号
                chain_id=x[4]
                pos_target.append((chain_id,dd,resid))
                
    tt=all_target
    new_target=all_target
    print('all_target的大小为：',len(all_target))
    print('tt的大小为：',len(tt))
    print('new_target的大小为：',len(new_target))
    for nu,i55 in enumerate(tt):
        if i55 in pos_target:
            new_target[nu]=1
        else:
            new_target[nu]=0
            
    #print(all_target)
    print('最后合并后的特征矩阵的大小为：',np.shape(final_feature))
    print('最后合并后的距离矩阵的大小为：',np.shape(combined_matrix))
    print('最后合并后的标签的大小为：',np.shape(new_target))
    if np.shape(final_feature)[0] != np.shape(new_target)[0]:
        print('有问题')
    #target_all=new_target
    input_file=os.path.join(save_npz_path,"%s.npz"%numm)
    np.savez(input_file,  H=final_feature, A1=combined_matrix, T=new_target)
    

1A0H_1_A_B
85
71
all_target的大小为： 156
tt的大小为： 156
new_target的大小为： 156
最后合并后的特征矩阵的大小为： torch.Size([156, 1310])
最后合并后的距离矩阵的大小为： (156, 156)
最后合并后的标签的大小为： (156,)
1A0H_1_D_E
81
71
all_target的大小为： 152
tt的大小为： 152
new_target的大小为： 152
最后合并后的特征矩阵的大小为： torch.Size([152, 1310])
最后合并后的距离矩阵的大小为： (152, 152)
最后合并后的标签的大小为： (152,)
1APY_1_C_D
63
73


/tmp/ipykernel_1929177/2362865335.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  esm_feature=torch.load(f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/prote

all_target的大小为： 136
tt的大小为： 136
new_target的大小为： 136
最后合并后的特征矩阵的大小为： torch.Size([136, 1310])
最后合并后的距离矩阵的大小为： (136, 136)
最后合并后的标签的大小为： (136,)
1APZ_1_C_D
65
73
all_target的大小为： 138
tt的大小为： 138
new_target的大小为： 138
最后合并后的特征矩阵的大小为： torch.Size([138, 1310])
最后合并后的距离矩阵的大小为： (138, 138)
最后合并后的标签的大小为： (138,)
1BCR_1_A_B
85
114
all_target的大小为： 199
tt的大小为： 199
new_target的大小为： 199
最后合并后的特征矩阵的大小为： torch.Size([199, 1310])
最后合并后的距离矩阵的大小为： (199, 199)
最后合并后的标签的大小为： (199,)
1BCR_1_B_A
115
86
all_target的大小为： 201
tt的大小为： 201
new_target的大小为： 201
最后合并后的特征矩阵的大小为： torch.Size([201, 1310])
最后合并后的距离矩阵的大小为： (201, 201)
最后合并后的标签的大小为： (201,)
1CE7_1_A_B
32
31
all_target的大小为： 63
tt的大小为： 63
new_target的大小为： 63
最后合并后的特征矩阵的大小为： torch.Size([63, 1310])
最后合并后的距离矩阵的大小为： (63, 63)
最后合并后的标签的大小为： (63,)
1CXP_1_A_D
10
9
all_target的大小为： 19
tt的大小为： 19
new_target的大小为： 19
最后合并后的特征矩阵的大小为： torch.Size([19, 1310])
最后合并后的距离矩阵的大小为： (19, 19)
最后合并后的标签的大小为： (19,)
1CXP_1_B_C
10
10
all_target的大小为： 20
tt的大小为： 20
new_target的大小为： 20
最后合并后的特征矩阵的大小为： torch.